Please ensure you have run all previous notebooks in sequence before running this.

#Model Building

In this lab we are going to wireup AMLS to our model.  

Note that:

* much of the AML code is, in effect, _scaffolding_ code.  We can use this to decorate any "pipeline" if we build a pattern

In [3]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (idna 2.1 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (idna 2.1 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (idna 2.1 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exception (idna 2.1 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (idna 2.1 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'}).
SDK version: 1.0.69

In [5]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mlops
Azure region: eastus
Subscription id: 52061d21-01dd-4f9e-aca9-60fff4d67ee2
Resource group: MLOpsWorkshop

In [6]:
#get the train and test datasets
train_data_path = "BikeBuyerTrain"
test_data_path = "BikeBuyerTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (3663, 12)
test: (1212, 12)
root
-- MaritalStatus: integer (nullable = true)
-- Gender: integer (nullable = true)
-- YearlyIncome: integer (nullable = true)
-- TotalChildren: integer (nullable = true)
-- NumberChildrenAtHome: integer (nullable = true)
-- Education: integer (nullable = true)
-- HouseOwnerFlag: integer (nullable = true)
-- NumberCarsOwned: integer (nullable = true)
-- CommuteDistance: integer (nullable = true)
-- Region: integer (nullable = true)
-- Age: integer (nullable = true)
-- BikeBuyer: integer (nullable = true)

#Define ML Pipeline

In [8]:
label = "BikeBuyer"
dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip"))
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

Now let's define a new model name, persist it, and save a history of the training runs.  We can create a series of training runs with different parameters and log them so we can evaluate them later.  

We'll save each model, feature set, and metrics as a separate child run.  

Looking at the next cell, what is the value we are actually changing in each run?

In [10]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil

model_name = "BikeBuyer_runHistory.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "BikeBuyer_Experiment")
root_run = myexperiment.start_logging()

# Regularization Rates
regs = np.arange(0.0, 1.0, 0.2)

# try a bunch of alpha values in a Linear Regression (Ridge) model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = LogisticRegression(regParam=reg)
        
        # put together the pipeline
        pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

        # train the model
        model_pipeline = pipe.fit(train)
        
        # make prediction
        pred = model_pipeline.transform(test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", train.columns)

        # save model
        model_pipeline.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        #run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0
Area under ROC: 0.6581025086124398
Area Under PR: 0.5482408921823594
Regularization rate: 0.2
Area under ROC: 0.6315622807839663
Area Under PR: 0.5200243021684962
Regularization rate: 0.4
Area under ROC: 0.6259418301823025
Area Under PR: 0.5140284062568512
Regularization rate: 0.6000000000000001
Area under ROC: 0.6238004356630225
Area Under PR: 0.5100655435282572
Regularization rate: 0.8
Area under ROC: 0.6223264519024047
Area Under PR: 0.5080016337123027
run id: 0898a6e2-fa6b-4650-b408-3a0d11704001

In [11]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}
child_run_metrics = {}

for r in root_run.get_children():
    child_runs[r.id] = r
    child_run_metrics[r.id] = r.get_metrics()


In [12]:
best_run_id = max(child_run_metrics, key = lambda k: child_run_metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', child_run_metrics[best_run_id])

Best run is: 79e5c580-54fa-47d2-8229-6367ba3b44ac
Metrics: {'columns': ['MaritalStatus', 'Gender', 'YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'Education', 'HouseOwnerFlag', 'NumberCarsOwned', 'CommuteDistance', 'Region', 'Age', 'BikeBuyer'], 'au_roc': 0.6581025086124398, 'reg': 0.0, 'au_prc': 0.5482408921823594}

What is the next cell showing me?  What could we name the columns to better convey the message?  How do we do that?

In [14]:
best_reg = child_run_metrics[best_run_id]['reg']
max_auc = child_run_metrics[best_run_id]['au_roc']

reg_auc = np.array([(child_run_metrics[k]['reg'], child_run_metrics[k]['au_roc']) for k in child_run_metrics.keys()])
reg_auc_sorted = reg_auc[reg_auc[:,0].argsort()]

import pandas as pd
df = pd.DataFrame(reg_auc_sorted)
spdf = spark.createDataFrame(df)
display(spdf)

0,1
0.0,0.6581025086124398
0.2,0.6315622807839663
0.4,0.6259418301823025
0.6000000000000001,0.6238004356630225
0.8,0.6223264519024047


In [15]:
#Download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

In [17]:
##unzip the model to dbfs (as load() seems to require that) and load it.
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_pipeline_best = PipelineModel.load(model_name)

In [18]:
# make prediction
pred = model_pipeline_best.transform(test)
output = pred[['MaritalStatus','Age','Gender','Region','Education','YearlyIncome','NumberCarsOwned','TotalChildren','NumberChildrenAtHome','HouseOwnerFlag','CommuteDistance','BikeBuyer']]
display(output.limit(5))

MaritalStatus,Age,Gender,Region,Education,YearlyIncome,NumberCarsOwned,TotalChildren,NumberChildrenAtHome,HouseOwnerFlag,CommuteDistance,BikeBuyer
1,39,1,2,1,10000,1,0,0,0,5,1
1,39,1,2,1,10000,2,0,0,0,1,0
1,39,1,2,1,10000,2,0,0,0,1,1
1,39,1,2,1,10000,2,0,0,0,1,1
1,39,1,2,1,10000,2,0,0,0,1,1


In [19]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.6581025086124398
Area Under PR: 0.5482408921823594

#Model Persistence

In [21]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_pipeline_best.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/BikeBuyer_runHistory.mml

In [22]:
%sh

ls -la /dbfs/BikeBuyer_runHistory.mml/*

/dbfs/BikeBuyer_runHistory.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Oct 23 19:37 .
drwxr-xr-x 1 root root 0 Oct 23 19:37 ..
-rw-r--r-- 1 root root 272 Oct 23 19:37 part-00000
-rw-r--r-- 1 root root 0 Oct 23 19:37 _SUCCESS

/dbfs/BikeBuyer_runHistory.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Oct 23 19:37 .
drwxr-xr-x 1 root root 0 Oct 23 19:37 ..
drwxr-xr-x 1 root root 0 Oct 23 19:37 0_StringIndexer_5e9c51fc4161
drwxr-xr-x 1 root root 0 Oct 23 19:37 1_VectorAssembler_116ee05a407d
drwxr-xr-x 1 root root 0 Oct 23 19:37 2_LogisticRegression_489fb920fc38

In [23]:
dbutils.notebook.exit("success")